# Driven systems and dynamical decoupling

Yi-Te Huang  
2025-07-10

Inspirations taken from an example in QuTiP-BoFiN article (Lambert et
al. 2023).

## Introduction

In this example, we show how to solve the time evolution with
time-dependent Hamiltonian problems in hierarchical equations of motion
approach.

Here, we study dynamical decoupling which is a common tool used to undo
the dephasing effect from the environment even for finite pulse
duration.

## Hamiltonian

We consider a two-level system coupled to a bosonic reservoir
($\textrm{b}$). The total Hamiltonian is given by
$H_{\textrm{T}}=H_\textrm{s}+H_\textrm{b}+H_\textrm{sb}$, where each
terms takes the form

$$
\begin{aligned}
H_{\textrm{s}}(t) &= H_0 + H_{\textrm{D}}(t),\\
H_0               &= \frac{\omega_0}{2} \sigma_z,\\
H_{\textrm{b}}    &=\sum_{k}\omega_{k}b_{k}^{\dagger}b_{k},\\
H_{\textrm{sb}}   &=\sigma_z\sum_{k}g_{\alpha,k}(b_k + b_k^{\dagger}).
\end{aligned}
$$

Here, $H_{\textrm{b}}$ describes a bosonic reservoir where $b_{k}$
$(b_{k}^{\dagger})$ is the bosonic annihilation (creation) operator
associated to the $k$th mode (with frequency $\omega_{k}$).

Furthermore, to observe the time evolution of the coherence, we consider
the initial state to be $$
ψ(t=0)=\frac{1}{\sqrt{2}}\left(|0\rangle+|1\rangle\right)
$$

Now, we need to build the system Hamiltonian and initial state with the
package
[`QuantumToolbox.jl`](https://github.com/qutip/QuantumToolbox.jl) to
construct the operators.

In [1]:
using HierarchicalEOM  # this automatically loads `QuantumToolbox`
using CairoMakie       # for plotting results

CairoMakie.activate!(type = "svg")

In [1]:
ω0 = 0.0
σz = sigmaz()
σx = sigmax()
H0 = 0.5 * ω0 * σz

# Define the operator that measures the 0, 1 element of density matrix
ρ01 = Qobj([0 1; 0 0])

ψ0 = (basis(2, 0) + basis(2, 1)) / √2


Quantum Object:   type=Ket()   dims=[2]   size=(2,)
2-element Vector{ComplexF64}:
 0.7071067811865475 + 0.0im
 0.7071067811865475 + 0.0im

The time-dependent driving term $H_{\textrm{D}}(t)$ has the form

$$
H_{\textrm{D}}(t) = \sum_{n=1}^N f_n(t) \sigma_x 
$$

where the pulse is chosen to have duration $\tau$ together with a delay
$\Delta$ between each pulses, namely

$$
f_n(t)
= \begin{cases}
  V & \textrm{if}~~(n-1)\tau + n\Delta \leq t \leq n (\tau + \Delta),\\
  0 & \textrm{otherwise}.
  \end{cases} 
$$

Here, we set the period of the pulses to be $\tau V = \pi/2$. Therefore,
we consider two scenarios with fast and slow pulses:

In [1]:
# a function which returns the amplitude of the pulse at time t
function pulse(p, t)
    τ = 0.5 * π / p.V
    period = τ + p.Δ

    if (t % period) < τ
        return p.V
    else
        return 0
    end
end

tlist = 0:0.4:400
amp_fast = 0.50
amp_slow = 0.01
delay = 20

fastTuple = (V = amp_fast, Δ = delay)
slowTuple = (V = amp_slow, Δ = delay)

# plot
fig = Figure()
ax = Axis(fig[1, 1], xlabel = L"t")
lines!(ax, tlist, [pulse(fastTuple, t) for t in tlist], label = "Fast Pulse", linestyle = :solid)
lines!(ax, tlist, [pulse(slowTuple, t) for t in tlist], label = "Slow Pulse", linestyle = :dash)

axislegend(ax, position = :rt)

fig

## Construct bath objects

We assume the bosonic reservoir to have a [Drude-Lorentz Spectral
Density](https://qutip.org/HierarchicalEOM.jl/stable/bath_boson/Boson_Drude_Lorentz/#Boson-Drude-Lorentz),
and we utilize the Padé decomposition. Furthermore, the spectral
densities depend on the following physical parameters:

-   the coupling strength $\Gamma$ between system and reservoir
-   the band-width $W$
-   the product of the Boltzmann constant $k$ and the absolute
    temperature $T$ : $kT$
-   the total number of exponentials for the reservoir $(N + 1)$

In [1]:
Γ = 0.0005
W = 0.005
kT = 0.05
N = 3
bath = Boson_DrudeLorentz_Pade(σz, Γ, W, kT, N)

HierarchicalEOM.BosonBath object with 4 terms.

## Construct HEOMLS matrix

> **Warning**
>
> Only provide the **time-independent** part of system Hamiltonian when
> constructing HEOMLS matrices (the time-dependent part `H_t` should be
> given when solving the time evolution).

In [1]:
tier = 6
M = M_Boson(H0, tier, bath)

Preparing block matrices for HEOM Liouvillian superoperator (using 4 threads)...
Progress: [                              ]   0.5% --- Elapsed Time: 0h 00m 01s (ETA: 0h 03m 29s)Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 01s (ETA: 0h 00m 00s)
Constructing matrix...[DONE]

Boson type HEOMLS matrix acting on even-parity ADOs
system dims = [2]
number of ADOs N = 210
data =
MatrixOperator(840 × 840)

## time evolution with time-independent Hamiltonian

In [1]:
noPulseSol = HEOMsolve(M, ψ0, tlist; e_ops = [ρ01]);

Solving time evolution for ADOs by Ordinary Differential Equations method...
Progress: [                              ]   0.1% --- Elapsed Time: 0h 00m 02s (ETA: 0h 33m 20s)Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 02s (ETA: 0h 00m 00s)

## Solve time evolution with time-dependent Hamiltonian

We need to provide a `QuantumToolbox.QuantumObjectEvolution` (named as
`H_D` in this case)

In [1]:
H_D = QobjEvo(σx, pulse)


Quantum Object Evo.:   type=Operator()   dims=[2]   size=(2, 2)   ishermitian=true   isconstant=false
ScalarOperator(0.0 + 0.0im) * MatrixOperator(2 × 2)

The keyword argument `params` in `HEOMsolve` will be passed to the
argument `p` in user-defined function (`pulse` in this case) directly:

In [1]:
fastPulseSol = HEOMsolve(M, ψ0, tlist; e_ops = [ρ01], H_t = H_D, params = fastTuple)
slowPulseSol = HEOMsolve(M, ψ0, tlist; e_ops = [ρ01], H_t = H_D, params = slowTuple)

Solving time evolution for ADOs by Ordinary Differential Equations method...
Progress: [                              ]   0.1% --- Elapsed Time: 0h 00m 02s (ETA: 0h 33m 20s)Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 02s (ETA: 0h 00m 00s)
Solving time evolution for ADOs by Ordinary Differential Equations method...
Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)

Solution of hierarchical EOM
(return code: Success)
----------------------------
Btier = 6
Ftier = 0
num_ados   = 1
num_expect = 1
ODE alg.: OrdinaryDiffEqLowOrderRK.DP5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}(OrdinaryDiffEqCore.trivial_limiter!, OrdinaryDiffEqCore.trivial_limiter!, static(false))
abstol = 1.0e-8
reltol = 1.0e-6

## Plot the coherence

In [1]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel = L"t", ylabel = L"\rho_{01}")
lines!(ax, tlist, real(fastPulseSol.expect[1, :]), label = "Fast Pulse", linestyle = :solid)
lines!(ax, tlist, real(slowPulseSol.expect[1, :]), label = "Slow Pulse", linestyle = :dot)
lines!(ax, tlist, real(noPulseSol.expect[1, :]),   label = "no Pulse",   linestyle = :dash)

axislegend(ax, position = :lb)

fig

## Version Information

In [1]:
HierarchicalEOM.versioninfo()

Lambert, Neill, Tarun Raheja, Simon Cross, Paul Menczel, Shahnawaz
Ahmed, Alexander Pitchford, Daniel Burgarth, and Franco Nori. 2023.
“QuTiP-BoFiN: A Bosonic and Fermionic Numerical
Hierarchical-Equations-of-Motion Library with Applications in
Light-Harvesting, Quantum Control, and Single-Molecule Electronics.”
*Phys. Rev. Res.* 5 (March): 013181.
<https://doi.org/10.1103/PhysRevResearch.5.013181>.